# Read Data from EIA API

The U.S. Energy Information Administration is committed to its free and open data by making it available through an Application Programming Interface (API) and its open data tools. EIA's API is multi-facetted and contains the following time-series data sets organized by the main energy categories.

## Libraries

### Installation

In [1]:
#pip install python-dotenv

### Import

In [2]:
#Import the required modules
import os
from dotenv import load_dotenv

import requests
import json

import pandas as pd
import numpy as np

import pickle

import csv

## Load API key from ".env"

In [3]:
#Load secret api_key from .env file
load_dotenv()

api_key = os.getenv('eia_api_key')
#print('eia_api_key: ', api_key)

## Specify EIA features to import

In [4]:
# List EAI Series IDs for the features
feature_list = {
    'WTIPUUS':'steo',
    'COPR_OPEC':'steo',
    'PAPR_NONOPEC':'steo',
    'COPRPUS':'steo',
    'PATC_OECD':'steo',
    'PATC_NON_OECD':'steo',
    'CICPIUS':'steo',
    'WPCPIUS':'steo',
    'WP57IUS':'steo',
    'ZOMNIUS':'steo',
    'PASC_OECD_T3':'steo',
    'PASXPUS':'steo',
    'COSQPUS':'steo',
    'COSXPUS':'steo',
    'MGWHUUS':'steo',
    'DSWHUUS':'steo',
    'BREPUUS':'steo',
    'WORL': 'international',
    'RNGWHHD': 'natural-gas'}

list(feature_list.keys())

['WTIPUUS',
 'COPR_OPEC',
 'PAPR_NONOPEC',
 'COPRPUS',
 'PATC_OECD',
 'PATC_NON_OECD',
 'CICPIUS',
 'WPCPIUS',
 'WP57IUS',
 'ZOMNIUS',
 'PASC_OECD_T3',
 'PASXPUS',
 'COSQPUS',
 'COSXPUS',
 'MGWHUUS',
 'DSWHUUS',
 'BREPUUS',
 'WORL',
 'RNGWHHD']

## Request to EIA API & store respose

In [5]:
def get_url_eia_api(feature_list, feature_id):
    
    if feature_list[feature_id] == 'steo':    
        url = 'https://api.eia.gov/v2/steo/data/?api_key=' + api_key +\
                '&frequency=monthly&data[0]=value&facets[seriesId][]='+ feature_id +\
                '&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000'
        description_field = 'seriesDescription'
        
    elif feature_list[feature_id] == 'international':
        url = 'https://api.eia.gov/v2/international/data/?api_key=' + api_key +\
                '&frequency=monthly&data[0]=value&facets[activityId][]=1'+\
                '&facets[productId][]=55&facets[countryRegionId][]=' + feature_id +\
                '&facets[unit][]=TBPD&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000'
        description_field = 'productName'
        
    elif feature_list[feature_id] == 'natural-gas':
        url = 'https://api.eia.gov/v2/natural-gas/pri/fut/data/?api_key=' + api_key +\
                '&frequency=monthly&data[0]=value&facets[series][]=' + feature_id +\
                '&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000'
        description_field = 'series-description'

    else:
        print('Feature ID not properly defined!')
        
    return url, description_field

In [6]:
# Initialize list; this is the final list that will store all the data from the json pull. 
read_data = []
feature_description = {}

# Pull in data via EIA API
for feature_id in list(feature_list.keys()):
    
    url, description_field = get_url_eia_api(feature_list, feature_id)
               
    r = requests.get(url)
    json_data = r.json()
    
    if r.status_code == 200:
        print('Success: ', feature_id)
    else:
        print('Error!')
    
    
    #get seriesDescription
    feature_description[feature_id] = json_data['response']['data'][0][description_field]
    print(feature_description[feature_id])
    
    # Cast data from the json pull to pandas dataframe
    df = pd.DataFrame(json_data['response']['data'], columns = ['period', 'value'])
    
    #rename columns
    df.columns = ['Date', feature_id]

    # Chage datetime and set as index
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m', errors="raise")
    df.set_index('Date', drop=True, inplace=True)
    
    display(df.head(3))
    
    # Concat the pandas dataframe into a list
    read_data.append(df)
    print('\n-------------------------')


Success:  WTIPUUS
West Texas Intermediate Crude Oil Price


,WTIPUUS
Date,
2024-12-01,72.0
2024-11-01,72.0
2024-10-01,72.0



-------------------------
Success:  COPR_OPEC
Crude Oil Production, OPEC Total


,COPR_OPEC
Date,
2024-12-01,28.880
2024-11-01,28.991
2024-10-01,29.202



-------------------------
Success:  PAPR_NONOPEC
Total non-OPEC liquids


...


-------------------------
Success:  WORL
Crude oil, NGPL, and other liquids


,WORL
Date,
2022-12-01,98009.234232
2022-11-01,99084.249563
2022-10-01,99141.569536



-------------------------
Success:  RNGWHHD
Henry Hub Natural Gas Spot Price (Dollars per Million Btu)


,RNGWHHD
Date,
2023-03-01,2.31
2023-02-01,2.38
2023-01-01,3.27



-------------------------


## Cast all data to single DataFrame

### Find the feature with most records to append all data

In [7]:
#find the longest dataseries
max_val = len(read_data[0])
max_idx = 0

# print field names and size for read_data
for i in range(0, len(read_data)):
    name = read_data[i].columns[0]
    size = len(read_data[i])
    phrase = str(i) + ': '+ name + ' (1*' + str(size) + ')'
    
    if size>max_val:
        max_val = size
        max_idx = i
    
    print(phrase)

0: WTIPUUS (1*420)
1: COPR_OPEC (1*384)
2: PAPR_NONOPEC (1*384)
3: COPRPUS (1*420)
4: PATC_OECD (1*420)
5: PATC_NON_OECD (1*420)
6: CICPIUS (1*420)
7: WPCPIUS (1*420)
8: WP57IUS (1*420)
9: ZOMNIUS (1*420)
10: PASC_OECD_T3 (1*264)
11: PASXPUS (1*420)
12: COSQPUS (1*420)
13: COSXPUS (1*420)
14: MGWHUUS (1*420)
15: DSWHUUS (1*420)
16: BREPUUS (1*420)
17: WORL (1*360)
18: RNGWHHD (1*315)


### Initiate a dataframe with biggest size feature
Using the feature with biggest number of records as the first field ensures that all rows can be appended to the the dataframe

In [8]:
#initiate the df with the the longest timeseries
df = read_data[max_idx]
df.index = df.index.astype('datetime64[ns]')
display(df.head(3))

,WTIPUUS
Date,
2024-12-01,72.0
2024-11-01,72.0
2024-10-01,72.0


### Merge_order all data frames

In [9]:
# Merge_order all data frames on date 
for i in range(0, len(read_data)):
    if i != max_idx:
        df_temp = read_data[i]
        df_temp.index = df_temp.index.astype('datetime64[ns]')
        df = pd.merge_ordered(df, df_temp, on = 'Date')

## Check the final dataframe

In [14]:
display(df.tail(3).T)

,417,418,419
Date,2024-10-01 00:00:00,2024-11-01 00:00:00,2024-12-01 00:00:00
WTIPUUS,72.0,72.0,72.0
COPR_OPEC,29.202,28.991,28.88
PAPR_NONOPEC,69.099035,69.352227,69.202363
COPRPUS,12.69427,12.84815,12.93507
PATC_OECD,46.39168,46.382377,46.983737
PATC_NON_OECD,55.492283,56.535749,57.552057
CICPIUS,3.128954,3.133612,3.138122
WPCPIUS,2.388834,2.391997,2.396435
WP57IUS,2.401871,2.359443,2.308539


## Store the data

In [11]:
# Change directory one step back and save as the root directory
ROOT_DIR = os.path.normpath(os.getcwd() + os.sep + os.pardir)

# Change direcotry to the 'data\raw' folder
path = ROOT_DIR + '\\data\\interim\\'

In [12]:
# Set a file name
file_data = path + 'eia_data.csv'
df.to_csv(file_data)
print('Save:\n', file_data)

Save:
 D:\gitProjects\WTI_Crude_Oil_Price_Prediction_Using_ML\data\interim\eia_data.csv


In [13]:
#save meta data
file_metadata = path + 'eia_data_description.csv'

# Open a CSV file for writing
with open(file_metadata, "w", newline="") as file:
    # Create a writer object
    writer = csv.writer(file)
    
    # Write the dictionary to the CSV file with two columns
    for key, value in feature_description.items():
        writer.writerow([key, value])
# Close the file
print('Save:\n', file_metadata)

Save:
 D:\gitProjects\WTI_Crude_Oil_Price_Prediction_Using_ML\data\interim\eia_data_description.csv
